In [2]:
import json

raw_ann = json.load(open('/data/datasets/ego4d_data/ego4d.json'))

In [6]:
import numpy as np
import pycolmap
from PIL import Image

# Extract SIFT features from an image

clip_info = raw_ann['clips']
clip_uid = '0021cb1c-2009-4469-b4c4-76829b9c1cda'
clip_info = [_info for _info in raw_ann['clips'] if _info['clip_uid'] == clip_uid][0]
video_uid = clip_info['video_uid']
video_info = [_info for _info in raw_ann['videos'] if _info['video_uid'] == video_uid][0]
video_meta = video_info['video_metadata']
display(clip_info['video_uid'])
display(video_info['device'])
display(f"{video_meta['display_resolution_width']} x {video_meta['display_resolution_height']}")

# cam parameters of GoPro Hero 4
# https://argus.web.unc.edu/camera-calibration-database/


'0836e1a4-11e6-4b31-bd39-f8e083fdadb3'

'GoPro Hero 4'

'1440 x 1080'

In [ ]:

img = Image.open(f'/data/datasets/ego4d_data/v2/vq2d_frames/520ss/{clip_uid}/frame_0000659.jpg')
width, height = img.size
# focal_length = 500.0  # example focal length
img = np.array(img.convert('L')).astype(np.float32) / 255.0
sift = pycolmap.Sift()
keypoints, descriptors = sift.extract(img)

# Perform absolute pose estimation
camera = pycolmap.Camera(
    model='SIMPLE_PINHOLE',
    width=width,
    height=height,
    # params=[focal_length, cx, cy]
)
points2D = np.array([[x1, y1], [x2, y2], ...])  # 2D pixel coordinates
points3D = np.array([[X1, y2, z1], [x2, y2, z2], ...])  # 3D world coordinates

result = pycolmap.absolute_pose_estimation(points2D, points3D, camera)

# Access estimation results
if result:
    qvec = result['qvec']  # Quaternion rotation
    tvec = result['tvec']  # Translation vector
    num_inliers = result['num_inliers']

# Perform pose refinement
inlier_mask = np.array([True, False, True, ...])  # Boolean mask of inliers
refined_result = pycolmap.pose_refinement(
    result['qvec'], result['tvec'],
    points2D, points3D, inlier_mask, camera
)

# Estimate fundamental matrix
points1 = np.array([[x1, y1], [x2, y2], ...])  # Points in image 1
points2 = np.array([[x1, y1], [x2, y2], ...])  # Corresponding points in image 2
F_result = pycolmap.fundamental_matrix_estimation(points1, points2)

if F_result:
    F = F_result['F']  # 3x3 fundamental matrix
    inliers = F_result['inliers']  # Inlier mask